In [1]:
import cv2
import torch
import gzip
import pickle
import torch.nn as nn
import numpy as np
from torch.utils.data import DataLoader
from torch.utils.data import TensorDataset
# from torch.utils.tensorboard import SummaryWriter

In [2]:
## 读取数据集
PATH = "./data/mnist/mnist.pkl.gz"
((x_train, y_train), (x_valid, y_valid), temp) = pickle.load(gzip.open(PATH, "rb"), encoding="latin-1")

In [3]:
## 数据格式转换
x_train, y_train, x_valid, y_valid = map(
    torch.tensor, (x_train, y_train, x_valid, y_valid)
)
y_train

tensor([5, 0, 4,  ..., 8, 4, 8])

In [4]:
## batch
batch_size = 64
train_data = TensorDataset(x_train, y_train)
train_loader = DataLoader(train_data, batch_size, shuffle=True)
test_data = TensorDataset(x_valid, y_valid)
test_loader = DataLoader(test_data, batch_size, shuffle=True)

In [5]:
# ## 借助tensorboard观察
# writer = SummaryWriter("logs")
# for epoch in range(64):
#     steps = 0
#     for data in train_loader:
#         imgs, targets = data
#         imgs.resize(28,28)
#         print(imgs.shape)
#         writer.add_images("Epoch: {}".format(epoch), imgs, steps)
#         steps += 1
# writer.close()

In [6]:
## 网络定义
class CNN(nn.Module):
    def __init__(self):
        super().__init__()
        self.hidden1 = nn.Linear(784, 128)
        self.hidden2 = nn.Linear(128, 256)
        self.out = nn.Linear(256, 10)

    def forward(self, x):
        x = nn.functional.relu(self.hidden1(x))
        x = nn.functional.relu(self.hidden2(x))
        x = self.out(x)
        return x

In [7]:
## 参数设置
# 模型例化
model = CNN()
# 训练次数
epochs = 50
# 学习率
lr = 0.01
# 优化器
optimizer = torch.optim.Adam(model.parameters(), lr=lr)
# 损失函数
loss = nn.CrossEntropyLoss()

In [8]:
## 开始训练
for epoch in range(epochs):
    losses = []
    val_loss = []
    model.train()
    for i, data in enumerate(train_loader):
        input_img, targets = data
        # 前向传播
        out = model(input_img)
        # 计算损失
        losses = loss(out, targets)
        # 反相传播
        losses.backward()
        # 更新参数
        optimizer.step()
        # 梯度清零
        optimizer.zero_grad()
    model.eval()
    for i, val_data in enumerate(test_loader):
        val_img, targets = val_data
        val_out = model(val_img)
        val_loss = loss(val_out, targets)
    print("当前step : {}/{} ,训练集损失 : {:.5f} , 验证集损失 : {:.5f}".format(epoch+1, epochs, losses, val_loss))

当前step : 1/50 ,训练集损失 : 0.16568 , 验证集损失 : 0.01874
当前step : 2/50 ,训练集损失 : 0.01112 , 验证集损失 : 0.00198
当前step : 3/50 ,训练集损失 : 0.00134 , 验证集损失 : 0.04457
当前step : 4/50 ,训练集损失 : 0.01291 , 验证集损失 : 0.35847
当前step : 5/50 ,训练集损失 : 0.00770 , 验证集损失 : 0.01023
当前step : 6/50 ,训练集损失 : 0.07823 , 验证集损失 : 0.16927
当前step : 7/50 ,训练集损失 : 0.08568 , 验证集损失 : 0.00385
当前step : 8/50 ,训练集损失 : 0.00369 , 验证集损失 : 0.56169
当前step : 9/50 ,训练集损失 : 0.00085 , 验证集损失 : 0.03413
当前step : 10/50 ,训练集损失 : 0.03892 , 验证集损失 : 0.00100
当前step : 11/50 ,训练集损失 : 0.08171 , 验证集损失 : 0.50068
当前step : 12/50 ,训练集损失 : 0.10462 , 验证集损失 : 1.99110
当前step : 13/50 ,训练集损失 : 0.07956 , 验证集损失 : 0.00004
当前step : 14/50 ,训练集损失 : 0.18930 , 验证集损失 : 0.00003
当前step : 15/50 ,训练集损失 : 0.00069 , 验证集损失 : 0.04326
当前step : 16/50 ,训练集损失 : 0.00086 , 验证集损失 : 0.00103
当前step : 17/50 ,训练集损失 : 0.14208 , 验证集损失 : 0.00001
当前step : 18/50 ,训练集损失 : 0.00004 , 验证集损失 : 0.00004
当前step : 19/50 ,训练集损失 : 0.00001 , 验证集损失 : 0.96519
当前step : 20/50 ,训练集损失 : 0.22887 , 验证集损失 : 1.43573
当前step : 

In [45]:
help(list.index)

Help on method_descriptor:

index(self, value, start=0, stop=9223372036854775807, /)
    Return first index of value.
    
    Raises ValueError if the value is not present.



In [47]:
## 结果预测
img = cv2.imread("./data/3.jpg",cv2.THRESH_BINARY)
cv2.normalize(img,img,0,255,cv2.NORM_MINMAX)
img = img / 255
img = cv2.resize(img,(28,28)).flatten()
img = torch.from_numpy(img)
predicted = model(img.to(torch.float32))
predicted = predicted.detach().numpy()
print(predicted)
predicted = predicted.tolist()
print(type(predicted))
print(predicted.index(max(predicted))+1)

[ 2.3945530e+01 -2.3413169e+00  6.5458656e+01  2.6385809e+01
 -5.3856440e+00 -8.7548275e+00  3.0709088e-02  1.7612244e+01
  1.1012868e+01 -5.2795574e+01]
<class 'list'>
3
